In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo as pm
import csv
import os
import pprint
import warnings
import scipy
import copy
import pprint
import sys
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
db_add = "<Your DB Address>"
auth_source = "<Auth Source>"
username = "<Your Username>"
password = "<Your Password>"

client=pm.MongoClient(db_add,
                      ssl=True,
                      username=username,
                      password=password,
                      authSource = auth_source,
                      authMechanism='SCRAM-SHA-1',
                      tlsAllowInvalidCertificates = True
                     )

db = client['carsharing'] #choosing DB

In [ ]:
ActiveBookings =db['ActiveBookings']
ActiveParkings = db['ActiveParkings']

PermanentBookings =db['PermanentBookings']
PermanentParkings =db['PermanentParkings']

enjoy_ActiveBookings = db['enjoy_ActiveBookings']
enjoy_ActiveParkings = db['enjoy_ActiveParkings']

enjoy_PermanentBookings = db['enjoy_PermanentBookings']
enjoy_PermanentParkings = db['enjoy_PermanentParkings']

In [ ]:
cities = ["Berlin","Firenze","Toronto"]
jan2018_start=int('1514764800')
jan2018_end=int('1517439600')

In [ ]:
########################## Number of Rentals ##########################

for c,city in enumerate(cities):
    bookings = PermanentBookings
    bookings = bookings.aggregate([
    {"$match": # Matches city for January
      {"$and": [{"city": city},
                {"init_time": {"$gte": jan2018_start}},
                {"init_time": {"$lte": jan2018_end}}
                ]}
      },
    {"$project": {         
        "_id": 0,
        "moved": {"$ne": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]},
        {"$arrayElemAt": ["$origin_destination.coordinates", 1]}]},
        "bookingDuration": {"$subtract": ["$final_time", "$init_time"]},
        "init_time" : ["$init_time"]
        }
      },
    {"$match": # Cars that moved and have booking duration more than 2 minutes and less than 3 hours
            {"$and": [{"moved": True},
            {"bookingDuration": {"$gte": 120}},
            {"bookingDuration": {"$lte": 10800}}
            ]}
      }
            ])
    
    temp=[]
    for booking in bookings:
        temp.append(booking)
    sortedBookings = sorted(temp, key = lambda i: i['init_time'][0])
    
    totHours = []
    for i in range(len(sortedBookings)):
        totHours.append(sortedBookings[i]['init_time'][0])
        
    counter=0
    counterPerHour = []
    out={}
    for i in range(0,744):
        init = (jan2018_start + (3600*i))
        end =  (init+3600)
        for j in totHours:
            if (j >= init and j < end):
                counter+=1
                out[i] = {}

        counterPerHour.append(counter)
        out[i] = {'unixTime': init,'hour': i, 'numberOfbookings': counter}
        counter = 0
        
    with open(f'Rentals_{c}.csv', 'w') as outfile:
        fields = ['unixTime', 'hour','numberOfbookings']
        write = csv.DictWriter(outfile, fieldnames=fields) 
        write.writeheader()
        for item in out.values():
            write.writerow(item)
            
    fig = plt.figure(1, figsize=(12, 6))
    x = np.linspace(1, 31, len(counterPerHour))
    plt.xticks(np.arange(32))
    plt.plot(x,counterPerHour,label=city)
    plt.grid(True)
    plt.legend()
    plt.ylabel("Number of Bookings")
    plt.xlabel("Day")
    plt.show()

In [ ]:
########################## Missing values management ##########################

if __name__ == '__main__':
    titles = os.listdir()
    
    #import cvs files
    for title in titles:
        if 'Rentals' in title:
            _ , title = title.replace('.csv','').split('_')
            title=int(title)
            if title == 0:
                title='Berlin'
                timeSeries = pd.read_csv('Rentals_0.csv')
                
            elif title == 1:
                title = "Firenze"
                timeSeries = pd.read_csv('Rentals_1.csv')
                
            elif title == 2:
                title = "Toronto"
                timeSeries = pd.read_csv('Rentals_2.csv')
                
            container=[]
            temp = 0
            
            # Check whether there are missing values and replace with 3 previous and next value's average which are non-zero
            timeSeries2=timeSeries.copy()
            for i in range(len(timeSeries2)-1):
                now = timeSeries2.iloc[i]['numberOfbookings']
                if now == 0:
                    temp+=1
                    if temp == 3:
                        for j in range(3):
                            container.append(timeSeries2.iloc[i-j-3]['numberOfbookings'])
                elif now !=0 and temp >= 3:
                    for j in range(3):
                        container.append(timeSeries2.iloc[i+j]['numberOfbookings'])
                        timeSeries2['numberOfbookings'][i-temp:i-1]=int(np.mean(container))
                        temp = 0
                        container=[]

            comparison = pd.DataFrame.equals(timeSeries2,timeSeries)
            timeSeries2['unixTime'] = pd.to_datetime(timeSeries2['unixTime'],infer_datetime_format=True,unit='s')
            indexedTimeSeries = timeSeries2.set_index(['hour'])

            # Plot time series
            fig = plt.figure(1, figsize=(12, 6))
            plt.grid(True)
            plt.plot(indexedTimeSeries['numberOfbookings'])
            plt.title(f'{title} - Missing data replaced')
            plt.xlabel('Time (hour)')
            plt.ylabel('Number of bookings')
            plt.savefig(f'{title} - Missing data replaced.jpg')
            plt.show()

            if comparison:
                print(f"No missing value found for {title}\n\n")
            indexedTimeSeries.to_csv(f'timeSeries_{title}.csv')

In [ ]:
########################## Stationary check ##########################

if __name__ == '__main__':
    titles = os.listdir()
    
    # Import cvs files
    for title in titles:
        if '.csv' in title:
            timeSeries = pd.read_csv(title)
            
            timeSeries = timeSeries.set_index(['hour'])
            timeSeries = timeSeries.drop('unixTime',axis=1)
            rolemean = timeSeries.rolling(window=24).mean()
            rolestd = timeSeries.rolling(window=24).std()
            print(rolemean,rolestd)
            
            # Rolling-statistics
            fig = plt.figure(1, figsize=(12, 6))
            original=plt.plot(timeSeries,color="blue",label="Original")
            mean=plt.plot(rolemean,color="red",label="Rolling Mean")
            std = plt.plot(rolestd, color="black", label="Rolling std")
            plt.grid(True)
            plt.xlabel('Time (hour)')
            plt.ylabel('Number of bookings')
            plt.legend(loc='best')
            city = title.replace('.csv','').split('_')
            plt.title(f'Rolling mean & standard deviation - {city}')
            plt.savefig(f'Rolling mean & standard deviation - {city}.jpg', dpi = 600)
            plt.show(block=False)
            
            # Dicky-Fuller test
            dftest = adfuller(timeSeries['numberOfbookings'],autolag='AIC')
            dfoutput= pd.Series(dftest[0:4],index=['Test Statistic','p-value','#Lags Used','Number of Observations'])
            
            for key,value in dftest[4].items():
                dfoutput[f'Critical value {key}'] = value
            print(dfoutput)

In [ ]:
########################## Convert Unix time to the Data for Berlin ##########################

series = pd.read_csv('Rentals_0.csv')
series_copy = copy.copy(series)
series_copy = series_copy.drop(series_copy.columns[1], axis=1)
series_copy.dropna(axis=0)
print(series)
series_copy['unixTime'] = pd.to_datetime(series_copy['unixTime'], unit='s')
print("Coverted Uninx time for Berlin", series_copy)
series_copy.to_csv("Berlin.csv", sep='\t', encoding='utf-8')

########################## Convert Unix time to the Data for Firenze ##########################

series2 = pd.read_csv('Rentals_1.csv')
series_copy2 = copy.copy(series2)
series_copy2 = series_copy2.drop(series_copy2.columns[1], axis=1)
series_copy2.dropna(axis=0)
print(series2)
series_copy2['unixTime'] = pd.to_datetime(series_copy2['unixTime'],unit='s')
print("Coverted Uninx time for Firenze", series_copy2)
series_copy2.to_csv("Firenze.csv",sep='\t', encoding='utf-8')

########################## Convert Unix time to the Data for Toronto ##########################

series3 = pd.read_csv('Rentals_2.csv')
series_copy3 = copy.copy(series3)
series_copy3 = series_copy3.drop(series_copy3.columns[1], axis=1)
series_copy3.dropna(axis=0)
print(series3)
series_copy3['unixTime'] = pd.to_datetime(series_copy3['unixTime'],unit='s')
print("Coverted Uninx time for Toronto", series_copy3)
series_copy3.to_csv("Toronto.csv",sep='\t', encoding='utf-8')

In [ ]:
########################## ACF-PACF computation ##########################

######### Plotting for Berlin #########
df = pd.read_csv("timeSeries_Berlin.csv", usecols=['unixTime','numberOfbookings'])
print(df)
plt.figure(figsize=(8,8))
pd.plotting.autocorrelation_plot(df.numberOfbookings)
plt.title('Berlin AutoCorrelation')
plt.savefig('Berlin AutoCorrelation.png', dpi=300)
plt.show()

fig1, ax1 = plt.subplots(1, 1)
plot_acf(df.numberOfbookings, ax = ax1, lags = 50)
plt.title('Berlin ACF zoom')
fig1.set_size_inches(8, 8)
plt.savefig('Berlin ACF zoom.png', dpi=300)

fig2, ax2 = plt.subplots( 1, 1)
plot_pacf(df.numberOfbookings, ax = ax2, lags = 50 )
plt.title('Berlin PACF')
fig2.set_size_inches(8, 8)
plt.savefig('Berlin PACF.png', dpi=300)
plt.grid()
plt.show()

######### Plotting for Firenze #########
df = pd.read_csv("timeSeries_Firenze.csv", usecols=['unixTime','numberOfbookings'])
plt.figure(figsize=(8,8))
pd.plotting.autocorrelation_plot(df.numberOfbookings)
plt.title('Firenze AutoCorrelation')
plt.savefig('Firenze AutoCorrelation.png', dpi=300)
plt.show()

fig1, ax1 = plt.subplots(1, 1)
plot_acf(df.numberOfbookings, ax = ax1, lags = 50)
plt.title('Firenze ACF zoom')
fig1.set_size_inches(8, 8)
plt.savefig('Firenze ACF zoom.png', dpi=300)

fig2, ax2 = plt.subplots( 1, 1 )
plot_pacf(df.numberOfbookings, ax = ax2, lags = 50 )
plt.title('Firenze PACF')
fig2.set_size_inches(8, 8)
plt.savefig('Firenze PACF.png', dpi=300)
plt.grid()
plt.show()

######### Plotting for Toronto #########
df = pd.read_csv("timeSeries_Toronto.csv", usecols=['unixTime','numberOfbookings'])
plt.figure(figsize=(8,8))
pd.plotting.autocorrelation_plot(df.numberOfbookings)
plt.title('Toronto AutoCorrelation')
plt.savefig('Toronto AutoCorrelation.png', dpi=600)
plt.show()

fig1, ax1 = plt.subplots(1, 1)
plot_acf(df.numberOfbookings, ax = ax1, lags = 50)
plt.title('Toronto ACF zoom')
fig1.set_size_inches(8, 8)
plt.savefig('Toronto ACF zoom.png', dpi=600)

fig2, ax2 = plt.subplots( 1, 1 )
plot_pacf(df.numberOfbookings, ax = ax2, lags = 50 )
plt.title('Toronto PACF')
fig2.set_size_inches(8, 8)
plt.savefig('Toronto PACF.png', dpi=600)
plt.grid()
plt.show()

In [ ]:
########################## Model training ##########################

p = 3 # p=3 estimated from PACF, as if we had q=0
q = 2 # More difficult to estimate, so we start with q=2
d = 0 # The time series is stationary

######### Plotting for Berlin #########
df = pd.read_csv("timeSeries_Berlin.csv", usecols=['unixTime','numberOfbookings'])

model = ARIMA(df.numberOfbookings,order=(p,d,q))
model_fit = model.fit(disp=True)

plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(df.numberOfbookings)
plt.plot(model_fit.fittedvalues)
plt.title("Real time series vs Predicted values - Berlin")
plt.legend(["real time series", "predicted values"])
step_hours = np.arange(0, df.shape[0], step=24)
step_ticks = df.iloc[step_hours]["unixTime"]
plt.xticks(np.arange(df.shape[0], step=24), step_ticks, rotation=45)
plt.xlabel("time [hours]")
plt.ylabel("number of rentals")
plt.savefig('OriginalPredicted_%s.png' % ("Berlin"))

residuals = pd.DataFrame(model_fit.resid)
residuals.plot(kind='kde', figsize=(15,5), grid=True, legend=False)

plt.title("Residuals for Berlin")
plt.xlabel("residual errors")
plt.savefig('error_%s.png' % ("Berlin"))

######### Plotting for Firenze #########
df2 = pd.read_csv("timeSeries_Firenze.csv", usecols=['unixTime','numberOfbookings'])

model2 = ARIMA(df2.numberOfbookings,order=(p,d,q))
model_fit2 = model2.fit(disp=True)

plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(df2.numberOfbookings)
plt.plot(model_fit2.fittedvalues)
plt.title("Real time series vs Predicted values - Firenze")
plt.legend(["real time series", "predicted values"])
step_hours = np.arange(0, df2.shape[0], step=24)
step_ticks = df2.iloc[step_hours]["unixTime"]
plt.xticks(np.arange(df2.shape[0], step=24), step_ticks, rotation=45)
plt.xlabel("time [hours]")
plt.ylabel("number of rentals")
plt.savefig('OriginalPredicted_%s.png' % ("Firenze"))

residuals2 = pd.DataFrame(model_fit2.resid)
residuals2.plot(kind='kde', figsize=(15,5), grid=True, legend=False)

plt.title("Residuals for Firenze")
plt.xlabel("residual errors")
plt.savefig('error_%s.png' % ("Firenze"))

######### Plotting for Toronto #########

df3 = pd.read_csv("timeSeries_Toronto.csv", usecols=['unixTime','numberOfbookings'])

model3 = ARIMA(df3.numberOfbookings,order=(p,d,q))
model_fit3 = model3.fit(disp=True)

plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(df3.numberOfbookings)
plt.plot(model_fit3.fittedvalues)
plt.title("Real time series vs Predicted values - Toronto")
plt.legend(["real time series", "predicted values"])
step_hours = np.arange(0, df3.shape[0], step=24)
step_ticks = df3.iloc[step_hours]["unixTime"]
plt.xticks(np.arange(df3.shape[0], step=24), step_ticks, rotation=45)
plt.xlabel("time [hours]")
plt.ylabel("number of rentals")
plt.savefig('OriginalPredicted_%s.png' % ("Toronto"))

residuals3 = pd.DataFrame(model_fit3.resid)
residuals3.plot(kind='kde', figsize=(15,5), grid=True, legend=False)

plt.title("Residuals for Toronto")
plt.xlabel("residual errors")
plt.savefig('error_%s.png' % ("Toronto"))

In [ ]:
def dataparse (time_in_secs):
    pd.datatime.fromtimestamp(float(time_in_secs))
    
df = pd.read_csv("timeSeries_Berlin.csv", usecols=['hour','numberOfbookings'])
df = df.rename(columns={'numberOfbookings':'rental','hour':'hour'})
df.rental += np.random.random_sample(len(df.rental)) / 10.0

df.hour -= df.hour[0]
# Replace method:
# [0] Replace with 0
# [1] Replace with previous value
# [2] Replace missing values assuming linear change
replace_method = 2
# Check missing values and print missing IDs
missing = False

for i in range(1,len(df)):
    if not df['hour'][i] == df['hour'][i-1] + 1:
        missing = True
        print("%i => %f, %i => %f" % (i-1,df['hour'][i-1],i,df['hour'][i]))
        start = int(df['hour'][i-1] + 1)
        end = int(df['hour'][i])
        startR = df['rental'][i-1]
        endR = df['rental'][i]
        
        deltaR = endR - startR
        deltaRoverH = deltaR / (end - start + 2)
        print (deltaRoverH)
        
        print("Missing hours from %i (%f) to %i (%f)" % (start, startR, end, endR))
        
        p = 1
        for k in range(start,end):
            if replace_method == 0:
                newval = 0
            elif replace_method == 1:
                newval = df['rental'][i-1]
            elif replace_method == 2:
                newval = startR + deltaRoverH * p
            else:
                print("Unknown replace method")
                
            # Add random noise
            newval = newval + np.random.random_sample() / 10.0
            
            df = df.append({ 'hour': k, 'rental' : newval }, ignore_index=True)
            print("New row (%i,%.3f)" % (k, newval))
            p = p + 1
            
df = df.sort_values(by=['hour'])
df = df.reset_index(drop=True)

df.plot(x = 'hour', y = 'rental', figsize=(15,5), title = "Berlin")
X = df.rental.values.astype(float)

# Define trainset size and window size
size = 24*7*2
test_len = 96

p_values = [0, 1, 2, 3, 4, 6, 8, 10, 12]
d_values = [0]
q_values = [0, 1, 2, 3, 4]

best_mse = float("inf")
best_r2 = -float("inf")
best_params = (0,0,0)

r2_scores = pd.DataFrame(None, columns=list(map(lambda x: 'p='+str(x),p_values)), 
                         index=list(map(lambda x: 'q='+str(x),q_values)))

In [ ]:
########################## GRID SEARCH ##########################

for p in p_values:
    for d in d_values:
        for q in q_values:
            print("Testing ARIMA model (%i,%i,%i)" % (p,d,q))
            # Split dataset into train and test
            train, test = X[0:size], X[size:(size+test_len)]
            
            history = [x for x in train]
            predictions = list()
            try:
                for t in range(0,test_len):
                    model = ARIMA(history, order=(p,d,q))
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    
                    yhat = output[0]
                    predictions.append(yhat)
                    
                    obs = test[t]
                    history.append(obs)
                    
                    # Sliding window: remove first element
                    history = history[1:]
                    
                mse = mean_squared_error(test, predictions)
                
                r2 = r2_score(test,predictions)
                r2_scores['p='+str(p)]['q='+str(q)] = r2
                print("R2\t%f" % r2)
                if r2 > best_r2:
                    best_r2 = r2
                print("MSE\t%f" % mse)
                if mse < best_mse:
                    best_mse = mse
                    best_params = (p,d,q)
                    
            except:
                print("\tError")
                continue
                
print("Best MSE %f with ARIMA(%i,%i,%i)" % (best_mse, best_params[0], best_params[1], best_params[2]))  

In [ ]:
########################## PLOT HEATMAP ##########################

errors = pd.DataFrame(r2_scores.to_numpy(dtype=float),
                      columns=list(map(lambda x: 'p='+str(x),p_values)),
                      index=list(map(lambda x: 'q='+str(x),q_values)))

grayscale = sns.cubehelix_palette(50, hue=1, rot=.5, light=.75, dark=.25, as_cmap=True)
ax = sns.heatmap(errors, linewidth=1, center=(errors.max().max()-errors.min().min())/2, 
                 cmap=grayscale, vmin = errors.min().min(), vmax = errors.max().max(), annot=True)
plt.savefig('heatmap_%s.png' % ("Berlin"), dpi = 600)

In [ ]:
########################## TESTS WITH SLIDING WINDOW ##########################

X = df.rental.values.astype(float)
# Define trainset size
sizes = range(3,22)# TWO WEEKS
# How many times I shift the window
test_len = 24*7
p = 6
d = 0
q = 1
r2_scores = []
for days in sizes:
    # Split dataset into train and test
    size = days*24
    train, test = X[0:size], X[size:(size+test_len)]
    
    history = [x for x in train]
    predictions = list()
    
    for t in range(0,test_len):
        try:
            model = ARIMA(history, order=(p,d,q))
            model_fit = model.fit(disp=0, maxiter=200)
            output = model_fit.forecast()
            
            yhat = output[0]
            predictions.append(yhat)
            
            obs = test[t]
            history.append(obs)
            
            # Sliding window
            history = history[1:]
            
        except:
            predictions.append(0)
            
    r2 = r2_score(test,predictions)
    r2_scores.append(r2)
    
    print("R2\t%f" % r2)
    if r2 > best_r2:
        best_r2 = r2
        
    print("MSE\t%f" % mse)
    if mse < best_mse:
        best_mse = mse
        best_params = (p,d,q)
        
labels=['3d',None,'5d',None,'1w',None,None,None,None,None,None,'2w',None,None,None,None,None,None,'3w']
indexes = []
nsp = 0
for label in labels:
    if label == None:
        nsp = nsp + 1
        s = ""
        for _ in range(0,nsp): s = s+" "
        indexes.append(s)
    else:
        indexes.append(label)

r2_df = pd.DataFrame(r2_scores,index=indexes)
plt.tick_params(labelsize=14)
plt.ylim((0,1))
plt.plot(r2_df)
plt.savefig('SLIDING_WINDOW.png')

########################## TESTS WITH EXPANDING WINDOW ##########################

X = df.rental.values.astype(float)
sizes = range(3,22)
test_len = 24*2 # 4 days
p = 6
d = 0
q = 1

r2_scores = []

for days in sizes:
    # Split dataset into train and test
    
    size = days*24
    train, test = X[0:size], X[size:(size+test_len)]
    
    history = [x for x in train]
    predictions = list()
    
    for t in range(0,test_len):
        try:
            model = ARIMA(history, order=(p,d,q))
            model_fit = model.fit(disp=0, maxiter=200)
            output = model_fit.forecast()
            
            yhat = output[0]
            predictions.append(yhat)
            
            obs = test[t]
            history.append(obs)
            #Expanding window!
            
        except:
            predictions.append(0)
            
    mse = mean_squared_error(test, predictions)
    r2 = r2_score(test,predictions)
    r2_scores.append(r2)
    
    print("R2\t%f" % r2)
    if r2 > best_r2:
        best_r2 = r2
        
    print("MSE\t%f" % mse)
    if mse < best_mse:
        best_mse = mse
        best_params = (p,d,q)
        
labels=['3d',None,'5d',None,'1w',None,None,None,None,None,None,'2w',None,None,None,None,None,None,'3w']
indexes = []
nsp = 0
for label in labels:
    if label == None:
        nsp = nsp + 1
        s = ""
        for _ in range(0,nsp): 
            s = s+" "
        indexes.append(s)
    else:
        indexes.append(label)
    
df = pd.DataFrame(r2_scores,index=indexes)
plt.tick_params(labelsize=14)
plt.ylim((0,1))
plt.plot(r2_df)
plt.savefig('EXPANDING_WINDOW.png')